# Import functions and the scattBO benchmark

In [1]:
from ase.lattice.cubic import FaceCenteredCubic, SimpleCubic, BodyCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked
from ase.cluster.icosahedron import Icosahedron
from ase.cluster.decahedron import Decahedron
from ase.cluster.octahedron import Octahedron
from debyecalculator import DebyeCalculator
import torch
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from utils.ScattBO import generate_structure, ScatterBO_small_benchmark, ScatterBO_large_benchmark

# Use a bayesian optimisation algorithm to minimise the difference between target scattering pattern and synthesis parameters - this example uses Dragonfly

In [2]:
from dragonfly import minimise_function

def benchmark_wrapper(params):
    """
    Wrapper function for the ScatterBO_small_benchmark function.

    Parameters:
    params (list): The parameters to pass to the ScatterBO_small_benchmark function.

    simulated_or_experimental (str): If 'simulated', use the filename 'Data/Gr/Target_PDF_small_benchmark.npy'. 
                                     If 'experimental', use the filename 'T2_0p7boro_15hrs_powder.npy'. Default is 'simulated'.
    
    scatteringfunction= (str): The scattering function to use. 'Gr' for pair distribution function, 'Sq' for structure factor. Default is 'Sq'.

    Returns:
    The result of the ScatterBO_small_benchmark function.
    """
    return ScatterBO_small_benchmark(params, plot=False, simulated_or_experimental='simulated', scatteringfunction='Gr')

# Define the domain for each parameter
domain = [
    [2, 12],  # pH values range from 2 to 12
    [20, 80],  # pressure values range from 20 to 80
    [0, 1]  # solvent can be 0 ('Ethanol') or 1 ('Methanol')
]

# Define the maximum capital
max_capital = 10

# Minimize the function
min_val, min_pt, history = minimise_function(benchmark_wrapper, domain, max_capital)

# Print the minimum value and the corresponding parameters
print ("min_val: ", min_val)
print ("min_pt: ", min_pt)

# Use the parameters that minimize the function as input to the generate_structure function
pH, pressure, solvent = min_pt
cluster = generate_structure(pH, pressure, solvent, atom='Au')
print ("Cluster type: ", cluster.structure_type)
print ("Number of atoms: ", len(cluster))

# Use the parameters that minimize the function as input to the ScatterBO_small_benchmark function
ScatterBO_small_benchmark((pH, pressure, solvent), plot=True, simulated_or_experimental='simulated', scatteringfunction='Gr')

Optimisation with bo(ei-ucb-ttei-add_ucb) using capital 10.0 (return_value)


/Users/ansoan/Documents/Materiale_Kemi/Postdoc/Research/Baysian_Optimisation/utils/ScattBO.py:37: UserWarning: Warning: Your system might have a CUDA-enabled GPU, but CUDA is not available. Computations will run on the CPU instead. For optimal performance, please install Pytorch with CUDA support. If you do not have a CUDA-enabled CPU, you can surpress this warning by specifying the 'device' argument as 'cpu'
  calc = DebyeCalculator(qmin=2, qmax=10.0, rmax=30, qstep=0.01)


Capital spent on initialisation: 5.0000(0.5000).
Legend: <iteration_number> (<num_successful_queries>, <fraction_of_capital_spent>):: curr_max=<current_maximum_value>, acqs=<num_times_each_acquisition_was_used>
#012 (011, 1.100):: curr_max=-0.67451, acqs=[ei:0, ucb:0, ttei:3, add_ucb:3], 
min_val:  0.6745078438588817
min_pt:  [2.00000105e+00 2.00000063e+01 1.76548290e-02]
Cluster type:  Octahedron
Number of atoms:  19


0.6745078438588817

# Now we try using a larger parameter space

In [3]:
from dragonfly import minimise_function

def benchmark_wrapper(params):
    """
    Wrapper function for the ScatterBO_small_benchmark function.

    Parameters:
    params (list): The parameters to pass to the ScatterBO_small_benchmark function.

    simulated_or_experimental (str): If 'simulated', use the filename 'Data/Gr/Target_PDF_small_benchmark.npy'. 
                                     If 'experimental', use the filename 'T2_0p7boro_15hrs_powder.npy'. Default is 'simulated'.
    
    scatteringfunction (str): The scattering function to use. 'Gr' for pair distribution function, 'Sq' for structure factor. Default is 'Sq'.

    Returns:
    The result of the ScatterBO_small_benchmark function.
    """
    return ScatterBO_large_benchmark(params, plot=False, simulated_or_experimental='simulated', scatteringfunction='Sq')

# Define the domain for each parameter
domain = [
    [0, 14],  # pH values range from 0 to 14
    [0, 100],  # pressure values range from 0 to 100
    [0, 3]  # solvent can be 0 ('Ethanol'), 1 ('Methanol'),  2 ('Water'), or 3 ('Other')
]

# Define the maximum capital
max_capital = 10

# Minimize the function
min_val, min_pt, history = minimise_function(benchmark_wrapper, domain, max_capital)

# Print the minimum value and the corresponding parameters
print ("min_val: ", min_val)
print ("min_pt: ", min_pt)

# Use the parameters that minimize the function as input to the generate_structure function
pH, pressure, solvent = min_pt
cluster = generate_structure(pH, pressure, solvent, atom='Au')
print ("Cluster type: ", cluster.structure_type)
print ("Number of atoms: ", len(cluster))

# Use the parameters that minimize the function as input to the ScatterBO_large_benchmark function
ScatterBO_large_benchmark((pH, pressure, solvent), plot=True, simulated_or_experimental='simulated', scatteringfunction='Sq')

Optimisation with bo(ei-ucb-ttei-add_ucb) using capital 10.0 (return_value)


/Users/ansoan/Documents/Materiale_Kemi/Postdoc/Research/Baysian_Optimisation/utils/ScattBO.py:37: UserWarning:




Capital spent on initialisation: 5.0000(0.5000).
Legend: <iteration_number> (<num_successful_queries>, <fraction_of_capital_spent>):: curr_max=<current_maximum_value>, acqs=<num_times_each_acquisition_was_used>
#012 (011, 1.100):: curr_max=-0.53878, acqs=[ei:2, ucb:2, ttei:1, add_ucb:1], 
min_val:  0.5387752562003624
min_pt:  [ 2.07442971 17.14423614  1.14807194]
Cluster type:  Octahedron
Number of atoms:  19


0.5387752562003624

# We can also try on experimental data

In [ ]:
from dragonfly import minimise_function

def benchmark_wrapper(params):
    """
    Wrapper function for the ScatterBO_small_benchmark function.

    Parameters:
    params (list): The parameters to pass to the ScatterBO_small_benchmark function.

    simulated_or_experimental (str): If 'simulated', use the filename 'Data/Gr/Target_PDF_small_benchmark.npy'. 
                                     If 'experimental', use the filename 'T2_0p7boro_15hrs_powder.npy'. Default is 'simulated'.
    
    scatteringfunction (str): The scattering function to use. 'Gr' for pair distribution function, 'Sq' for structure factor. Default is 'Sq'.

    Returns:
    The result of the ScatterBO_small_benchmark function.
    """
    return ScatterBO_large_benchmark(params, plot=False, simulated_or_experimental='experimental', scatteringfunction='Sq')

# Define the domain for each parameter
domain = [
    [0, 14],  # pH values range from 0 to 14
    [0, 100],  # pressure values range from 0 to 100
    [0, 3]  # solvent can be 0 ('Ethanol'), 1 ('Methanol'),  2 ('Water'), or 3 ('Other')
]

# Define the maximum capital
max_capital = 10

# Minimize the function
min_val, min_pt, history = minimise_function(benchmark_wrapper, domain, max_capital)

# Print the minimum value and the corresponding parameters
print ("min_val: ", min_val)
print ("min_pt: ", min_pt)

# Use the parameters that minimize the function as input to the generate_structure function
pH, pressure, solvent = min_pt
cluster = generate_structure(pH, pressure, solvent, atom='Au')
print ("Cluster type: ", cluster.structure_type)
print ("Number of atoms: ", len(cluster))

# Use the parameters that minimize the function as input to the ScatterBO_large_benchmark function
ScatterBO_large_benchmark((pH, pressure, solvent), plot=True, simulated_or_experimental='experimental', scatteringfunction='Sq')